In [6]:
import warnings 
warnings.filterwarnings(action='ignore')
import os
import base64
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import imageio
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
from keras import backend as K
import colorsys
import random
from io import BytesIO
from keras.models import load_model
import urllib

In [7]:
import json

In [8]:
def lambda_handler(image, mode):
    
    if mode is 'url':
        input_image = Image.open(urllib.request.urlopen(image))
    if mode is 'base64':

        # Encode
        with open(image, 'rb') as binary_file:
            binary_file_data = binary_file.read()
            base64_encoded_data = base64.b64encode(binary_file_data)
            
        # Decode
        with open('decoded_image.png', 'wb') as file_to_save:
            decoded_image_data = base64.decodebytes(base64_encoded_data)
            file_to_save.write(decoded_image_data)
            input_image = Image.open('decoded_image.png')
            
    sess = K.get_session()
    model = load_model('./model_data/yolov2.h5')
    features = model.output
    anchors = read_anchors('./model_data/yolo_anchors.txt')
    class_names = read_classes('./model_data/coco_classes.txt')
    yolo_outputs = features_to_boxes(features, anchors, len(class_names))
    
    output_image_64_encoded, out_scores, out_boxes, out_classes = predict(sess, input_image, yolo_outputs, model, class_names)

#     return json.dumps({
#         "out_scores": out_scores,
#         "out_boxes": out_boxes,
#         "out_classes": out_classes
#     })
    
    return output_image_64_encoded, out_scores, out_boxes, out_classes


In [9]:
def preprocess_image(input_image, model_image_size):
    
    #image = Image.open(urllib.request.urlopen(url))
        
    # Retrieve image shape (necessary to scale predicted bounding boxes later on)
    image_shape = input_image.size[::-1]
    
    num_channel = len(input_image.split())
    
    input_image = input_image.convert("RGB")
    
    # Resize image to correspond to model image size with BICUBIC interpolation mode (cubic spline interpolation)
    resized_image = input_image.resize(model_image_size, Image.BICUBIC)
    
    # Convert image to a numpy array
    image_data = np.array(resized_image, dtype='float32')
    
    # Convert if has 4 channels
    if image_data.shape[2] == 4:
        image_data = cv2.cvtColor(input_image, cv2.COLOR_BGRA2BGR)
        
    # Normalize image
    image_data /= 255.
    
    # Add batch dimension 
    image_data = np.expand_dims(image_data, 0)  
    
    # Display the image 
    plt.show()
    
    return image_data, image_shape

In [10]:
def read_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

def read_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def features_to_boxes(features, anchors, num_classes):
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params
    anchors_tensor = K.reshape(K.variable(anchors), [1, 1, 1, num_anchors, 2])
    # Dynamic implementation of conv dims for fully convolutional model
    conv_dims = K.shape(features)[1:3]  # assuming channels last
    # In YOLO the height index is the inner most iteration
    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])
    conv_width_index = K.tile(
        K.expand_dims(conv_width_index, 0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_index, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(features))
    features = K.reshape(
        features, [-1, conv_dims[0], conv_dims[1], num_anchors, num_classes + 5])
    conv_dims = K.cast(K.reshape(conv_dims, [1, 1, 1, 1, 2]), K.dtype(features))
    box_xy = K.sigmoid(features[..., :2])
    box_wh = K.exp(features[..., 2:4])
    box_confidence = K.sigmoid(features[..., 4:5])
    box_class_probs = K.softmax(features[..., 5:])
    # Adjust preditions to each spatial grid point and anchor size.
    # Note: YOLO iterates over height index before width index.
    box_xy = (box_xy + conv_index) / conv_dims
    box_wh = box_wh * anchors_tensor / conv_dims
    return box_xy, box_wh, box_confidence, box_class_probs

def yolo_evaluate(yolo_outputs, image_shape=(720., 1280.), max_output_size=10, score_threshold=.6, iou_threshold=.5):
    # Unpack bounding box params 
    xy, wh, confidence, class_probs = yolo_outputs
    # Convert boxes information from (middle-point-coordinates, width-height) to corners coordinates 
    boxes = boxes_to_corners(xy, wh)
    # Step 1: eliminate boxes with low probabilities - filter boxes with scores lower than the threshold
    scores, boxes, classes = filter_boxes(confidence, boxes, class_probs, score_threshold)
    # Step 2: eliminate boxes with non-max scores and high overlap - filter boxes with IoU lower than the threshold
    scores, boxes, classes = non_max_suppression(scores, boxes, classes, max_output_size, iou_threshold)
    # Scale boxes to match input image size
    boxes = scale_boxes(boxes, image_shape)
    return scores, boxes, classes

def boxes_to_corners(box_xy, box_wh):
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)
    return K.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]])  # x_max

def filter_boxes(confidence, boxes, class_probs, threshold = 0.6):
    # Compute the score for each box (object probability multiplied by class probability)
    scores = confidence * class_probs
    # Find class with maximum score for each box and respective score
    classes_with_max_score = K.argmax(scores, axis=-1)
    max_scores = K.max(scores, axis=-1)
    # Define a mask (binary filter) to discard boxes with score lower than the set threshold
    mask = (max_scores >= threshold)
    # Apply the mask to discard boxes data with the score lower than the set threshold
    scores = tf.boolean_mask(max_scores, mask)
    boxes =  tf.boolean_mask(boxes, mask)
    classes = tf.boolean_mask(classes_with_max_score, mask)
    return scores, boxes, classes

def non_max_suppression(scores, boxes, classes, max_output_size = 10, iou_threshold = 0.5):
    # Initialize max_output_size variable (we need it later to perform tf.image.non_max_suppression)
    max_output_size_tensor = K.variable(max_output_size, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_output_size_tensor]))
    # Return indices from the boxes data to keep: eliminate non-max scores with a significant overlap with the max score
    remaining_indices = tf.image.non_max_suppression(boxes, scores, max_output_size_tensor, iou_threshold=0.5)
    # Get rid of bounding box data with non-max scores and significant overlap with max score 
    scores = K.gather(scores, remaining_indices)
    boxes = K.gather(boxes, remaining_indices)
    classes = K.gather(classes, remaining_indices)
    return scores, boxes, classes

def scale_boxes(boxes, image_shape):
    # Retrieve image dimensions
    height = image_shape[0]
    width = image_shape[1]
    # Stack and reshape image dimensions to correspond to boxes coordinates (b_x, b_y, b_h, b_w)
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    image_dims = K.cast(image_dims, 'float32')
    # Normalize boxes by image dimensions
    boxes = boxes * image_dims
    return boxes

def generate_colors(class_names):
    hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
    # Fixed seed for consistent colors across runs
    random.seed(10101)  
    # Shuffle colors to decorrelate adjacent classes
    random.shuffle(colors)  
    # Reset seed to default
    random.seed(None)  
    return colors


In [11]:
def draw_boxes(image, out_scores, out_boxes, out_classes, class_names):
    # Generate colors for drawing predicted bounding boxes
    colors = generate_colors(class_names)
    # Load a font - workaround for ImageFont.truetype OSError
    file = open("./font/FiraMono-Medium.otf", "rb")
    bytes_font = BytesIO(file.read())
    font = ImageFont.truetype(bytes_font, size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    file.close()
    # Set bounding box line thickness
    thickness = (image.size[0] + image.size[1]) // 300
    for i, c in reversed(list(enumerate(out_classes))):
        # Retrieve box parameters
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        # Create output label
        label = '{} {:.2f}'.format(predicted_class, score)
        # Draw the bounding box on the image
        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)
        # Retrieve box coordinates
        top, left, bottom, right = box
        # Limit bounding box coordinates to the image dimensions
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        # Set origin for the label text
        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])
        # My kingdom for a good redistributable image drawing library.
        for i in range(thickness):
            draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colors[c])
        draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colors[c])
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw

In [12]:
def predict(sess, input_image, yolo_outputs, model, class_names):
    # Preprocess input images to match model input size 
    image_data, image_shape = preprocess_image(input_image, model_image_size = (608, 608))

    # Optimize: perform forward propagation 
    out_scores, out_boxes, out_classes = sess.run(yolo_evaluate(yolo_outputs, image_shape), feed_dict={model.input: image_data,
                                                                                                 K.learning_phase(): 0})
    # Draw bounding boxes on the input image 
    draw_boxes(input_image, out_scores, out_boxes, out_classes, class_names)
    # Save edited image
    #path, filename = os.path.split(img_path)
    filename = 'output_image.jpg'
    #input_image.save(os.path.join('./images/test_out', filename), quality=90)
    
    # Display the image 
#     output_image = imageio.imread(os.path.join('./images/test_out', filename))
#     imshow(output_image)
    
    # Convert to base64 
#     output_image = open(os.path.join('./images/test_out/', filename), 'rb') #open binary file in read mode
#     image_read = output_image.read()
#     output_image_64_encoded = base64.encodestring(image_read)
    
    return input_image, out_scores, out_boxes, out_classes


In [20]:
url = 'https://agendasorocaba.com.br/wp-content/uploads/2019/07/McDonalds-02.jpg'
test_image = './images/person.png'
image, v, v, v = lambda_handler(image=test_image, mode='base64')

In [23]:
image_string = b"7Vtbc9o4FP41zOw+hLFlG8Mjl9B2Jp3tbDrTx4ywhVFjW15bBOivX8mSL7JMgMSENIV2BuucY13O+c5FQulZ02j7KYXJ6ivxUdgDhr/tWbMeAKZhjNgXp+wExbEdQQhS7EuhinCPf6HiTUldYx9liiAlJKQ4UYkeiWPkUYUG05RsVLElCdVRExggjXDvwVCn/sA+XUmqORhVjM8IBys59BC4grGA3mOQknUsx+sBa55/BDuCRV9yodkK+mRTI1m3PWuaEkLFU7SdopDrtlCbeG++h1vOO0UxPeaFx/H913+cJXlcL+6+p3GaETe4kWvJ6K7QB/KZemSTpHRFAhLD8LaiTvI1I96rwVqVzB0hCSOajPgTUbqTtoZrShhpRaNQcjMKUzrmtmMEL4RZhr2CPMdhISamxuezd8WSlJF16qFnlmmW+mY4RiRCNN2x91IUQoqf1P6hBFRQypWvfiOYjQwMCX7LlVCX0C8gU/TA1hMgKl+qLMNWDnc1sYQLZPuHAbbRNozsjT2IDotWbYUVKcfCCbgwwcWBEZMYNVBhnAEVLNzkZnpGDnSBnlOtbgHV6uao4d6nyZ8JJmLIJxiupRqmIUYS7nXwkDUNcYymZRDnplySmE5JSNJcxmL/5nzsSZBCn/dS8EokpOQRtb2wZPCo0Zf5h9F9mK1KPD6hlGIW9+/gAoXfSIYpJjHjLQilJKoJjEMccAblqJ1A2fLYhFCq4pUvQIKZOYxsy0XzIWGWiNUu8ZbPY8JyQMKZ0Tbg2bQPN5ndT5FA6hePz2fCmuJJlfKEYgsH4JNF2+ddQIdsEVHMBlZc2d5UGXAgSata7itobSCvAe0FOLL+gHADjgw3nSSr/eFGMVQHMQBoMWD87QsjfIIUbeCu21DAiyzHdu1RjTfDKetIOHPMYaBHhKlrWua8LYZ8yFgBE/wQSPV3EjCalY45MLWA4Q71gFHQOg8YIx10EfzFNAGM2/l955gbGGPLck/DHHBdtov4YzCHltkDC56xD1O/G9A5AzVLWaaepd4UdOa72S19X+G4ZceEYr+RzhjlTLXzscXz6BLFszMaKdABw+eL5wPyZyqenYvD6ZVFT+c7ascZNjYxo76jdiKQqW2qta5sxzrU1cn7865LJ5Yergg4ZDbjZfbXoWS80vp7xjHaJ7x3Xs/LnyfUAH2jfnHk/Xmp7HXFh27C8Q9W6Rp3MFr4sPOSd+4Obw37tJJ3ZjhT0319ySsq22a9WxbCWsn7lnVuKNSt4Drk85+Uvwc0zq0El6Q+ShucFUkx27RQqHRUU0S5wjOU0xffwwH7iKAUhjjJuK42K0zRfQJzV94we5xigiZAyt9ijG50yz59dYdcpLGacot6r65c0z6Xci1NuaamXbZgqqqRgR7/govKc1gMz6Ti2jyv6aMR9v08V8i0yYZxJj1n1ogVrSe6kqhmmdPs2okpVTdx9E0naDkaBec6GgXDD+MlltkofQb2hZ1EP0b6XXU7MM1GBAKuHt/fVLuW8WG0a5l2U7vmhQO8dUxJ/3to1zHsd6bbgaZbXbnX5NnmJ4drzDfNnparWVI/dbtass0nzYZPXrgOsj5OHaQdAV26DrIcTbdshoOQe0WWwFjR8uC/Nb+yNvGE0sb8BCRYwL+Y4dl/NgGj9elv/sjVZ3D13ixhhMOdeD0iMclyWykiwuG4gJFsq3HZUyC+nea1RIetmVPzS3Vlq9CBk2uBUWb8mU/M4at2mOoOyZqlbGHmF3UDqm6EqtsGEJouOXlwYi0BX04282YVojjNyGllmKrERKjibRmsOFENV5wpAxZnFgeeYgIO17MzyzlV6OLM3Fk4uR6+FEYRwqoJtvqd8k7pe/kSQE6raU243TF2KVmlSSpHdLgrVpJOZUNjV9FHNbLwzMqQNZb00FYezCp6UBu+iZe8WYKmTlShLOU0zAtXZb4vvLVwkGueOyIWv7OKxdYPbuyrJY/ZARvqDz5gMLywJfWKRf+972rJtl0EeF+1p3PEXvt6re3819qiJMTLXbMifx6fHeDRNlQ8Di8NR3AYjoWacZT//UvdZO1GPoiNPWqugy0fbFxYvNX8cj6zFaX8D3zGXBNg7vmx3cfM5EvMAk/a99iIYO5DCtkXp7M6ah6SgGQ3LD3OBzxpz/9F0KP9n9kDZ9yYYNhP4qAJjnOEp6EKh8IcF4ODXjGUlyHv9YvVXVzFP+6qoxR+dyEoT4wovX1CIj+a+8LSYu09Ivrm8aZ529bS443bcjzwgh9qe3KDUbtsUm0trNv/AQ=="


In [21]:
buffered = BytesIO()
image.save(buffered, format="PNG")
img_str = base64.b64encode(buffered.getvalue())
img_base64 = bytes("data:image/png;base64,", encoding='utf-8') + img_str

In [22]:
img_str

b'...'

In [24]:
img_base64 = bytes("data:image/png;base64,", encoding='utf-8') + image_string

In [29]:
img_base64.decode('utf-8')

''

In [30]:
decoded_binary_image = base64.b64decode(img_base64) 
image_file = BytesIO(decoded_binary_image)
input_image = Image.open(image_file)

Error: Incorrect padding